# About PyTorch

Unlike some other frameworks like TensorFlow, PyTorch uses dynamic computation graphs which allow for greater flexibility and debugging capabilities. The key benefits of PyTorch include:

* Simple and intuitive Python API for building neural networks
* Broad support for GPU/TPU acceleration
* Built-in support for automatic differentiation
* Distributed training capabilities
* Interoperability with other Python libraries like NumPy

PyTorch Lightning is a lightweight wrapper built on top of PyTorch that further simplifies the process of researcher workflow and model development. With Lightning, data scientists can focus more on designing models rather than boilerplate code. Key advantages of Lightning include:

* Provides structure to organize PyTorch code
* Handles training loop boilerplate code
* Accelerates research experiments with hyperparameters tuning
* Simplifies model scaling and deployment

NOTE: This doesn't run because not all elements of the lightning module are defined.
See: https://lightning.ai/docs/pytorch/stable/data/datamodule.html#setup 

In [1]:
import torch
# import lightning

## Building a Model with Pytorch
PyTorch uses tensors, similar to NumPy arrays, as its core data structure. Tensors can be operated on by GPUs and support automatic differentiation for building neural networks.

The model below defines a convolutional neural network with two convolutional layers and three fully connected layers for classifying 10 classes. The forward() method defines how data passes through the network.

In [9]:
# defining a simple neural network for image classification


# import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

## Train the Model with Lightning

In [14]:
import lightning as pl

class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = Net()
    
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)
    
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=64)
        
model = LitModel()

In [15]:
trainer = pl.Trainer()
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NameError: name 'train_dataloader' is not defined

In [ ]:
# save model results
result = trainer.test(model, test_dataloader)
print(result)

In [ ]:
# hyperparameter tuning
tuner = pl.Tuner(trainer)
tuner.fit(model, train_dataloader)
print(tuner.results)

In [ ]:
# adjusting for overfitting
model = LitModel()
model.add_module('dropout', nn.Dropout(0.2)) # Regularization

trainer = pl.Trainer(early_stop_callback=True) # Early stopping

In [ ]:
# Save
trainer.save_checkpoint("model.ckpt") 

# Load
model = LitModel.load_from_checkpoint(checkpoint_path="model.ckpt")